## Face Recognition – Unlock Your Computer With Your Face!


### Step 1 - Create Training Data

In [ ]:
import cv2
import numpy as np

In [1]:
import cv2
import numpy as np

# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Initialize Webcam
def captureImages(user,path):
    count = 0
    cap=cv2.VideoCapture('http:/192.168.43.1:8080/video')
    # Collect 100 samples of your face from webcam input
    print(path)
    while True:

        ret, frame = cap.read()
        if not ret:
            print("Photo not clicked")
            continue
        if face_extractor(frame) is not None:
            count += 1
            face = cv2.resize(face_extractor(frame), (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            # Save file in specified directory with unique name
            file_name_path = path+ user + str(count) + '.jpg'
            print(file_name_path)
            cv2.imwrite(file_name_path, face)

            # Put count on images and display live count
            cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Cropper', face)

        else:
            print("Face not found")
            pass

        if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
            break

    print("Collecting Samples Complete")
    cv2.destroyAllWindows()          
    cap.release()

<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-1-b50d482c18ad>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


### Step 2 - Train Model

In [2]:
import cv2
import numpy as np
from os import listdir,mkdir
from os.path import isfile, join, exists

# Get the training data we previously made
data_path = './users/'
print(data_path)
if not exists(data_path):
    mkdir(data_path)
print(listdir())

captureImages(input("Enter User 1 ID (Numerical): "),data_path)

captureImages(input("Enter User 2 ID (Numerical): "),data_path)

onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]
print(cv2.__version__)
# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    lid=int(onlyfiles[i].split('.')[0])
    Labels.append(lid)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

# Initialize facial recognizer
# model = cv2.face.createLBPHFaceRecognizer()
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()
# pip install opencv-contrib-python
# model = cv2.createLBPHFaceRecognizer()

vimal_model  = cv2.face_LBPHFaceRecognizer.create()
# Let's train our model 
vimal_model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully")


./users/
['.ipynb_checkpoints', '50_Startups.csv', 'churn_model.ipynb', 'Churn_Modelling.csv', 'cnn-cat-dog-model (1).h5', 'cnn-model.h5', 'CNN.ipynb', 'diabetes_model.ipynb', 'emailsender.py', 'face', 'Face Recognition – TASK-6.ipynb', 'haarcascade_frontalface_default.xml', 'marks.csv', 'marks.pk1', 'mgr.py', 'multi linear regression model.ipynb', 'myweightheight.h5', 'pima-indians-diabetes.data.csv', 'pywhatkit_dbs.txt', 'setup.py', 'simple linear regression model.ipynb', 'single_prediction', 'temp.py', 'testing_set', 'titanic binary classification.ipynb', 'titanic_test.csv', 'titanic_train.csv', 'training_set', 'Untitled.ipynb', 'users', 'weight-height.csv', 'weight-height.ipynb']
Enter User 1 ID (Numerical): 1
./users/
Face not found
Face not found
Face not found
Face not found
./users/11.jpg
./users/12.jpg
./users/13.jpg
./users/14.jpg
./users/15.jpg
./users/16.jpg
./users/17.jpg
./users/18.jpg
./users/19.jpg
./users/110.jpg
./users/111.jpg
./users/112.jpg
./users/113.jpg
./users/

### Step 3 - Run Our Facial Recognition

In [4]:
import cv2
import numpy as np
import os
import pywhatkit as pkt
import getpass
import boto3,time
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture('http://192.168.43.1:8080/video')
flag=False
while True:

    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        results = vimal_model.predict(face)
        # harry_model.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            user = str(results[0])[0]
            display_string = str(confidence) + '% Confident it is User ' + user
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        if confidence > 80 and user=='1':
            cv2.putText(image, "Hey User 1", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image ) 
            
            pkt.sendwhatmsg_instantly("+917073973656","Hlo,Task Completed")
            print("whatapp message send successfully")
            p = getpass.getpass()
            pkt.send_mail("purusoni220@gmail.com",p,"test mail","Task completed","rahulbairwabhl@gmail.com")
            
            break
        elif confidence > 80 and user=='2':
            cv2.putText(image, "Hey User 2", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
            
            print("Launching EC2 Instance....")
            
            client=boto3.client("ec2")
            ec2_id = client.run_instances(
                ImageId="ami-0ad704c126371a549",
                MinCount=1,
                MaxCount=1,
                InstanceType="t2.micro",
                KeyName="AWS-KEY",
                SubnetId="subnet-10e00b7b",
                SecurityGroupIds=["sg-02b328240cd51089d"]

            )
            print("Instance Launched successfully...")
            print("Creating EBS volume of 5 GB...")
            
            volume_id = client.create_volume( AvailabilityZone='ap-south-1a',
                  Size=5,
                  VolumeType='gp2',
                  TagSpecifications=[
                      {
                          'ResourceType': 'volume',
                          'Tags': [
                              {
                                  'Key': 'Name',
                                  'Value': 'volume_task'
                              },
                          ]
                      },
                  ],)
            
            print("Please Wait! While Instance is Launching.....")
            time.sleep(30)
            
            instanceid=ec2_id["Instances"][0]["InstanceId"]
            vid = volume_id["VolumeId"]
            print(instanceid,vid)
            print("Volume  created successfully...\n")
            print(" Attaching EBS volume to EC2 instance...")
            client.attach_volume(
                        Device='/dev/xvdb',
                        InstanceId=instanceid,
                        VolumeId = vid            
                        ) 
            print("volume attached successfully...")
            break
        else:
            cv2.putText(image, "I dont know, how r u", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )
            
    except Exception as e:
        #print(e)
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
    if cv2.waitKey(1) == 13: #13 is the Enter Key
            break
cap.release()
cv2.destroyAllWindows()     

<>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-4-35cbf59a3037>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


whatapp message send successfully
········
Email sent Successfully!
